# This is the code to compute distance to features using GDAL.

- Inputs are:
    - Coastline
    - Administrative units.
- Process:
    - Get the spatial parameters from the administrative units
    - Rasterize the coastline with the AU parameters.
    - Calculate raster distance
    - Mask the shapefile
- Author: Rubén Crespo Ceballos


In [1]:
from osgeo import gdal
import os
import numpy as np
import pyproj
import rasterio
from rasterio.transform import from_origin
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon
from rasterio import features
from rasterio.mask import mask

In [6]:
def get_spatial_parameters_from_dataframe(gdf, tolerance, resolution):
    # Get the bounds of the GeoDataFrame
    xmin, ymin, xmax, ymax = gdf.total_bounds

    # Apply tolerance to extend the bounds
    xmin -= tolerance
    ymin -= tolerance
    xmax += tolerance
    ymax += tolerance

    # Calculate the number of pixels in x and y directions
    cols = int((xmax - xmin) / resolution)
    rows = int((ymax - ymin) / resolution)

    # Create a transform for the raster
    transform = from_origin(xmin, ymax, resolution, resolution)
    return rows, cols, transform

def rasterize_geodataframe_to_1_and_0(gdf, rows, cols, transform, output_path):

    # Create an empty array to hold the rasterized values
    rasterized_array = np.zeros((rows, cols), dtype=np.uint8)

    # Rasterize the geometry
    shapes = gdf.geometry
    rasterized_array = features.rasterize(
        shapes=shapes,
        out_shape=(rows, cols),
        transform=transform,
        fill=0,  # fill value for pixels outside polygons
        all_touched=True,
        dtype=np.uint8,
    )

    # Set pixels inside the polygons to 1
    rasterized_array[rasterized_array > 0] = 1

    # Create the raster as a Rasterio dataset
    raster = rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=rows,
        width=cols,
        count=1,
        dtype=np.uint8,
        crs=gdf.crs,
        transform=transform,
        nodata=0,  # Set the nodata value
        compress='deflate'  # Compression method
    )

    # Write the raster array to the dataset
    raster.write(rasterized_array, 1)
    raster.close()

    return output_path  # Return the rasterized array for further processing

def compute_distance_raster(input_path, output_path, cols, rows, epsg):
    crs = pyproj.CRS.from_epsg(epsg)

    output_ds = gdal.Open(input_path, 0)
    band = output_ds.GetRasterBand(1) # Get raster information
    geotransform = output_ds.GetGeoTransform() # Set the Geotransformation
    
    # Create an empty raster where we will write the distance value once it is computed:
    driver = gdal.GetDriverByName('GTiff') # Type of data (GeoTiff)
    out_ds = driver.Create(output_path, cols, rows, 1, gdal.GDT_Float32) # Create the raster in the path, with 'width' and 'height' and type of data (previously defined)
    out_ds.SetGeoTransform(geotransform) # Define the Geotransformation to the created raster
    out_ds.SetProjection(crs.to_wkt()) # Set the Coordinate System
    out_band = out_ds.GetRasterBand(1) # Get raster info

    gdal.ComputeProximity(band, out_band, ['VALUES=1']) # Compute distances
    out_ds.FlushCache() # Refresh dataset
    return output_path

def mask_raster_with_shapefile(raster_path, shapefile_path, output_dir, epsg):
    # Read the raster dataset
    with rasterio.open(raster_path) as src:
        # Read the shapefile
        gdf = gpd.read_file(shapefile_path)
        gdf = gdf.dissolve()
        gdf = holes_eraser(gdf, epsg)

        for idx, feature in gdf.iterrows():
            # Get the geometry of the feature
            geo_feature = gpd.GeoSeries(feature['geometry'])

            # Crop the raster to the bounding box of the feature
            out_image, out_transform = mask(src, geo_feature, crop=True)

            # Update metadata for the cropped raster
            out_meta = src.meta.copy()
            out_meta.update({
                'height': out_image.shape[1],
                'width': out_image.shape[2],
                'transform': out_transform,
                'nodata' : 0,
                'compress': 'deflate',  # Set compression to deflate
                'tiled': True,  # Enable tiling
            })

            # Write the cropped raster to a new file
            with rasterio.open(output_dir, 'w', **out_meta) as dest:
                dest.write(out_image)

def holes_eraser(gdf, epsg):
    multipolygon_wkt = gdf['geometry']
    multipolygon = wkt.loads(str(multipolygon_wkt[0]))

    list_parts = []

    for polygon in multipolygon.geoms:
        list_interiors = []
        for interior in polygon.interiors:
            list_interiors.append(interior)
        
        # Check if there are interior rings
        if list_interiors:
            # Create a new polygon with the exterior rings
            temp_pol = Polygon(polygon.exterior)
            list_parts.append(temp_pol)

        else:
            # If there are no interior rings, just append the polygon without modifications
            list_parts.append(polygon)
        
    new_multipolygon = MultiPolygon(list_parts)
    no_holes_gdf = gpd.GeoDataFrame(geometry=[new_multipolygon])
    
    # Set the CRS for the GeoDataFrame
    crs = pyproj.CRS.from_epsg(epsg) 
    no_holes_gdf.crs = crs
    return no_holes_gdf

In [ ]:
# Inputs
vector_coastline = r"Z:\z_resources\im-nca-germany\boundaries\destat_germany_boundaries_adm_lev0_corrected_4326.shp"
vector_administrateive_units = r"Z:\z_resources\im-nca-germany\boundaries\destat_germany_boundaries_adm_lev1_4326.shp"
output_path = r"Z:\z_resources\im-nca-germany\boundaries"
resolution = 0.002 # Resolution in meters/degrees depending your projection
tolerance = 3
epsg = 4326 # Projection of all the inputs 

In [ ]:
output_raster_path = os.path.join(output_path, os.path.basename(vector_coastline).replace("shp","tif"))
output_distance_path = os.path.join(output_path, os.path.basename(vector_coastline).replace(".shp","_distance.tif"))  
final_output_raster_path = os.path.join(output_path, os.path.basename(vector_coastline).replace(".shp","_final.tif")) 

In [3]:
#Open the data
coastline_gdf = gpd.read_file(vector_coastline)
coastline_gdf = coastline_gdf.dissolve()
administrateive_units_gdf = gpd.read_file(vector_administrateive_units)
administrateive_units_gdf = administrateive_units_gdf.dissolve()

# Get the spatial parameters from the administrative units
rows, cols, transform = get_spatial_parameters_from_dataframe(administrateive_units_gdf, tolerance, resolution)
#At rasterizing, apply the spatial parameters of the adminsitratative units
raster_path = rasterize_geodataframe_to_1_and_0(coastline_gdf, rows, cols, transform, output_raster_path)
raster_distance_path = compute_distance_raster(raster_path, output_distance_path, cols, rows, epsg)
mask_raster_with_shapefile(raster_distance_path, vector_administrateive_units, final_output_raster_path, epsg)